## PIP INSTALL

In [4]:
!pip install torch torchvision torchaudio

  Using cached torch-1.13.1-cp37-cp37m-win_amd64.whl (162.6 MB)
     ---------------------------------------- 1.1/1.1 MB 33.8 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 32.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install requests beautifulsoup4 gensim nltk spacy transformers
  


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
!pip install bert-embedding


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached bert_embedding-1.0.1-py3-none-any.whl (13 kB)
  Using cached mxnet-1.4.0-py2.py3-none-win_amd64.whl (21.9 MB)
  Using cached gluonnlp-0.6.0-py3-none-any.whl
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)


In [35]:
!python -m spacy download fr_core_news_lg

     -------------------------------------- 571.8/571.8 MB 3.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip uninstall -y numpy

Found existing installation: numpy 1.21.6
Uninstalling numpy-1.21.6:
  Successfully uninstalled numpy-1.21.6


In [17]:
!pip install numpy

  Using cached numpy-1.21.6-cp37-cp37m-win_amd64.whl (14.0 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bert-embedding 1.0.1 requires numpy==1.14.6, but you have numpy 1.21.6 which is incompatible.
mxnet 1.4.0 requires numpy<1.15.0,>=1.8.2, but you have numpy 1.21.6 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import spacy
nlp = spacy.load("fr_core_news_lg") 



C:\Users\andy\anaconda3\envs\Data\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [7]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

## Scraping

In [8]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.winamax.fr/poker/faq.php?topic="
topics = range(0, 6)
all_faqs = {}

for topic in topics:
    url = base_url + str(topic)
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')       
        questions = soup.select('div.title')
        answers = soup.select('div.answer p')
        for q, a in zip(questions, answers):
            all_faqs[q.text.strip()] = a.text.strip()




In [9]:
import pandas as pd

QA_df = pd.DataFrame(list(all_faqs.items()), columns=['questions', 'answers'])
QA_df

,questions,answers
0,Comment compléter mon inscription ?,Pour compléter votre inscription et accéder au...
1,Mes documents ont été validés mais mon compte ...,Une fois votre compte validé (pièce d’identité...
2,J’ai renseigné une date de naissance incorrect...,"Afin que vous puissiez vous connecter, nous vo..."
3,Je n'ai pas reçu mon email d'activation,Il se peut qu'un logiciel de type anti-spam ai...
4,J'ai oublié mon mot de passe,"Pour vous connecter à Winamax, vous devez indi..."
...,...,...
59,L’un des paris de mon MyMatch a été annulé,Si une sélection est annulée à l’intérieur d’u...
60,Les paris sportifs permettent-ils de remporter...,"Lorsque vous placez des paris sportifs, vous c..."
61,Que sont les Freebets ?,Les Freebets sont des paris gratuits offerts p...
62,Comment fonctionnent les paris système ?,Les paris système vous permettent de parier en...


In [10]:
## Data Preprocessing
import re
class TextPreprocessor():
    def __init__(self, data_df, column_name=None):
        self.data_df = data_df  
        if not column_name and type(colum_name) == str:
            raise Exception("column name is mandatory. Make sure type is string format")
        self.column = column_name
        self.convert_lowercase()    
        self.applied_stopword = False
        self.processed_column_name = f"processed_{self.column}"
        
    def convert_lowercase(self):
        ## fill empty values into empty
        self.data_df.fillna('',inplace=True)
        ## reduce all the columns to lowercase
        self.data_df = self.data_df.apply(lambda column: column.astype(str).str.lower(), axis=0)    

    def remove_question_no(self):
        ## remove question no        
        self.data_df[self.column] = self.data_df[self.column].apply(lambda row: re.sub(r'^\d+[.]',' ', row))    
        
    def remove_symbols(self):
        ## remove unwanted character          
        self.data_df[self.column] = self.data_df[self.column].apply(lambda row: re.sub(r'[^A-Za-z0-9\sàâäéèêëïîôöùûüÿç]', ' ', row))    

    def remove_stopwords(self):
        ## remove stopwords and create a new column 
        for idx, question in enumerate(self.data_df[self.column]):      
            self.data_df.loc[idx, self.processed_column_name] = remove_stopwords(question)        

    def apply_lemmatization(self, perform_stopword):
        ## get the root words to reduce inflection of words 
        lemmatizer = WordNetLemmatizer()    
        ## get the column name to perform lemma operation whether stopwords removed text or not
        if perform_stopword:
            column_name = self.processed_column_name
        else:
            column_name = self.column
        ## iterate every question, perform tokenize and lemma
        for idx, question in enumerate(self.data_df[column_name]):

            lemmatized_sentence = []
            ## use spacy for lemmatization
            doc = nlp(question.strip())
            for word in doc:       
                lemmatized_sentence.append(word.lemma_)      
                ## update to the same column
                self.data_df.loc[idx, self.processed_column_name] = " ".join(lemmatized_sentence)

    def process(self, perform_stopword = True):
        self.remove_question_no()
        self.remove_symbols()
        if perform_stopword:
            self.remove_stopwords()
        self.apply_lemmatization(perform_stopword)    
        return self.data_df

In [11]:
## pre-process training question data

text_preprocessor = TextPreprocessor(QA_df.copy(), column_name="questions")
processed_QA_df = text_preprocessor.process(perform_stopword=True)
processed_QA_df.head(10)

,questions,answers,processed_questions
0,comment compléter mon inscription,pour compléter votre inscription et accéder au...,comment compléter mon inscription
1,mes documents ont été validés mais mon compte ...,une fois votre compte validé (pièce d’identité...,mon document avoir être valider mais mon compt...
2,j ai renseigné une date de naissance incorrect...,"afin que vous puissiez vous connecter, nous vo...",j avoir renseigner un date naissance incorrect...
3,je n ai pas reçu mon email d activation,il se peut qu'un logiciel de type anti-spam ai...,je ne avoir pas recevoir mon e-mail d activation
4,j ai oublié mon mot de passe,"pour vous connecter à winamax, vous devez indi...",j avoir oublier mon mot passer
5,j ai oublié le mot de passe de mon compte wina...,"dans ce cas, veuillez contacter le support en ...",j avoir oublier le mot passer mon compte winam...
6,je voudrais modifier mon mot de passe,connectez-vous sur le site avec votre email et...,je vouloir modifier mon mot passer
7,je souhaite modifier l adresse email enregistr...,pour changer votre adresse email d'identificat...,je souhaiter modifier l adresse e-mail enregis...
8,je souhaite changer mon pseudo,connectez-vous sur le site à l’aide de vos ide...,je souhaiter changer mon pseudo
9,je souhaite modifier mon avatar,"il vous suffit de cliquer le bouton ""mon compt...",je souhaiter modifier mon avatar


In [12]:
#from bert_embedding import BertEmbedding

In [13]:
class TF_IDF():
    def __init__(self):
        self.dictionary = None    
        self.model = None
        self.bow_corpus = None

    def create_tf_idf_model(self, data_df, column_name):
        ## create sentence token list
        sentence_token_list = [sentence.split(" ") for sentence in data_df[column_name]]

        ## dataset vocabulary
        self.dictionary = Dictionary(sentence_token_list) 

        ## bow representation of dataset
        self.bow_corpus = [self.dictionary.doc2bow(sentence_tokens) for sentence_tokens in sentence_token_list]

        ## compute TF-IDF score for corpus
        self.model = TfidfModel(self.bow_corpus)

        ## representation of question and respective TF-IDF value
        print(f"First 10 question representation of TF-IDF vector")
        for index, sentence in enumerate(data_df[column_name]):
            if index <= 10:
                print(f"{sentence} {self.model[self.bow_corpus[index]]}")
            else:
                break

    def get_vector_for_test_set(self, test_df, column_name):
        ## store tf-idf vector
        testset_tf_idf_vector = []
        sentence_token_list = [sentence.split(" ") for sentence in test_df[column_name]]
        test_bow_corpus = [self.dictionary.doc2bow(sentence_tokens) for sentence_tokens in sentence_token_list]   
        for test_sentence in test_bow_corpus:
            testset_tf_idf_vector.append(self.model[test_sentence])      

        return testset_tf_idf_vector

    def get_training_QA_vectors(self):
        QA_vectors = []
        for sentence_vector in self.bow_corpus:
            QA_vectors.append(self.model[sentence_vector])      
        return QA_vectors

    def get_train_vocabulary(self):
        vocab = []
        for index in self.dictionary:
            vocab.append(self.dictionary[index])
        return vocab

In [14]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import torch

In [16]:
from transformers import BertTokenizer, BertModel

In [17]:
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

'conda' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [18]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-french-europeana-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-french-europeana-cased")




C:\Users\andy\anaconda3\envs\Data\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\andy\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at dbmdz/bert-base-french-europeana-cased were not used when initializing BertModel: ['cls.seq_relations

In [31]:
def get_bert_embeddings(sentences):
    all_tokens = []
    all_embeddings = []
    
    for sentence in sentences:
        # Tokenize input text
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        
        # Get BERT embeddings
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Extract tokens and embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])  # Assuming batch size of 1
        embeddings = outputs.last_hidden_state[0].tolist()
        
        all_tokens.append(tokens)
        all_embeddings.append(embeddings)
    
    # Pair tokens with embeddings for each sentence
    bert_output = list(zip(all_tokens, all_embeddings))
    return bert_output



In [32]:
tf_idf = TF_IDF()
tf_idf.create_tf_idf_model(processed_QA_df, "processed_questions")
## get the tf-idf reprentation 
question_QA_vectors = tf_idf.get_training_QA_vectors()

First 10 question representation of TF-IDF vector
comment compléter mon inscription [(0, 0.21878517146541068), (1, 0.6863715158615069), (2, 0.6863715158615069), (3, 0.09960588925764022)]
mon document avoir être valider mais mon compte être bloquer [(3, 0.13580810698690746), (4, 0.22070726519916853), (5, 0.4679181971749496), (6, 0.1632339964987991), (7, 0.4679181971749496), (8, 0.38993183097912465), (9, 0.4679181971749496), (10, 0.31194546478329976)]
j avoir renseigner un date naissance incorrect et je ne pouvoir pas connecter [(4, 0.17824626524673423), (11, 0.31491438449082665), (12, 0.377897261388992), (13, 0.2519315075926613), (14, 0.377897261388992), (15, 0.2316555500223097), (16, 0.07511621812724871), (17, 0.377897261388992), (18, 0.14483292215222995), (19, 0.15210600952153247), (20, 0.31491438449082665), (21, 0.377897261388992), (22, 0.17824626524673423)]
je ne avoir pas recevoir mon e-mail d activation [(3, 0.08233441498928427), (4, 0.26760999718242456), (16, 0.11277572011711878)

In [33]:
## Get the document vocabulary list from TF-IDF
document_vocabulary = tf_idf.get_train_vocabulary()

In [34]:
question_QA_bert_embeddings_list = get_bert_embeddings(processed_QA_df["questions"].to_list())

In [35]:
## helps to retrieve similar question based of input vectors/embeddings for test query
def retrieveSimilarFAQ(train_question_vectors, test_question_vectors, train_QA_df, train_column_name, test_QA_df, test_column_name):
    for test_index, test_vector in enumerate(test_question_vectors):
        # Store top 5 similar questions - tuple of (similarity_score, train_index)
        top_5_similar = [(-1, -1) for _ in range(5)]
        
        for train_index, train_vector in enumerate(train_question_vectors):
            sim_score = cosine_similarity(train_vector, test_vector)[0][0]
            
            # Check if this score is higher than the smallest in top_5
            if sim_score > top_5_similar[0][0]:
                top_5_similar[0] = (sim_score, train_index)
                # Sort the top 5 by score to always have the smallest score at 0 index
                top_5_similar.sort(key=lambda x: x[0])

        print("######")
        print(f"Query Question: {test_QA_df[test_column_name].iloc[test_index]}")    
        for _, sim_Q_index in top_5_similar:
            print(f"Retrieved Question: {train_QA_df[train_column_name].iloc[sim_Q_index]}")
        print("######")


In [36]:
test_query_string = ["Je voudrais annuler un pari que je viens de placer", 
                     "Je n’ai pas reçu mon argent de mon pari gagnant",
                     "changer de mot de passe",
                "Comment s'inscrire aux tournois gratuits ?",
                "Je souhaite retirer mes gains ",
                "Je ne parviens pas à me connecter"]

test_QA_df = pd.DataFrame(test_query_string, columns=["test_questions"])              
## pre-process testing QA data
text_preprocessor = TextPreprocessor(test_QA_df, column_name="test_questions")
query_QA_df = text_preprocessor.process(perform_stopword=True)

In [37]:
## TF-IDF vector represetation
query_QA_vectors = tf_idf.get_vector_for_test_set(query_QA_df, "processed_test_questions")
query_QA_df.head()

,test_questions,processed_test_questions
0,je voudrais annuler un pari que je viens de pl...,je vouloir annuler pari que je venir placer
1,je n ai pas reçu mon argent de mon pari gagnant,je ne avoir pas recevoir mon argent mon pari g...
2,changer de mot de passe,changer mot passer
3,comment s inscrire aux tournois gratuits,comment s inscrire à tournoi gratuit
4,je souhaite retirer mes gains,je souhaiter retirer mon gain


In [38]:
query_QA_bert_embeddings_list = get_bert_embeddings(test_QA_df["test_questions"].to_list())

In [39]:
## store QA bert embeddings in list
question_QA_bert_embeddings = []
for embeddings in question_QA_bert_embeddings_list:
    question_QA_bert_embeddings.append(embeddings[1])

## store query string bert embeddings in list
query_QA_bert_embeddings = []
for embeddings in query_QA_bert_embeddings_list:
    query_QA_bert_embeddings.append(embeddings[1])

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
retrieveSimilarFAQ(question_QA_bert_embeddings, query_QA_bert_embeddings, processed_QA_df, "questions", query_QA_df, "test_questions")

######
Query Question: je voudrais annuler un pari que je viens de placer
Retrieved Question: je voudrais modifier mes informations personnelles
Retrieved Question: je voudrais créer un autre compte
Retrieved Question: je voudrais créditer mon compte
Retrieved Question: je voudrais modifier mon mot de passe
Retrieved Question: je voudrais annuler un pari que je viens de placer
######
######
Query Question: je n ai pas reçu mon argent de mon pari gagnant
Retrieved Question: j ai reçu un bonus mais il n a pas été crédité sur mon compte
Retrieved Question: je n ai pas reçu mon retrait
Retrieved Question: je souhaite faire un dépôt via paypal
Retrieved Question: je voudrais créditer mon compte
Retrieved Question: je n ai pas reçu les gains de mon pari gagnant
######
######
Query Question: changer de mot de passe
Retrieved Question: je souhaite modifier l adresse email enregistrée sur mon compte
Retrieved Question: comment rejoindre une partie de poker  
Retrieved Question: comment compléte